In [1]:
from mindspore import context
from mindspore.nn import Adam
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge import UpdaterMD
from mindsponge import WithEnergyCell
from mindsponge.control import VelocityVerlet, Langevin
from mindsponge.potential import SphericalRestrict
from mindsponge.function import VelocityGenerator
from mindsponge.callback import WriteH5MD, RunInfo

[WARNING] ME(312679:139849280619776,MainProcess):2023-03-25-16:48:28.669.607 [mindspore/run_check/_check_version.py:75] Can not found cuda libs, please confirm that the correct cuda version has been installed, you can refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')

In [4]:
system.reduplicate([0.3, 0, 0])
system.reduplicate([0, 0.3, 0])
new_sys = system.copy([0, 0, -0.3])
system.reduplicate([0, 0, 0.3])
system.append(new_sys)

Molecule<>

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = Adam(system.trainable_params(), 1e-3)

In [7]:
mini = Sponge(system, potential, opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [8]:
run_info = RunInfo(10)
mini.run(1000, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-03-25 16:48:39
[MindSPONGE] Step: 0, E_pot: 242.99136
[MindSPONGE] Step: 10, E_pot: 163.5599
[MindSPONGE] Step: 20, E_pot: 93.392136
[MindSPONGE] Step: 30, E_pot: 16.301376
[MindSPONGE] Step: 40, E_pot: -66.25972
[MindSPONGE] Step: 50, E_pot: -136.00613
[MindSPONGE] Step: 60, E_pot: -183.78194
[MindSPONGE] Step: 70, E_pot: -219.24011
[MindSPONGE] Step: 80, E_pot: -250.27698
[MindSPONGE] Step: 90, E_pot: -276.4144
[MindSPONGE] Step: 100, E_pot: -299.6914
[MindSPONGE] Step: 110, E_pot: -326.58896
[MindSPONGE] Step: 120, E_pot: -357.65543
[MindSPONGE] Step: 130, E_pot: -390.4199
[MindSPONGE] Step: 140, E_pot: -421.48557
[MindSPONGE] Step: 150, E_pot: -449.73285
[MindSPONGE] Step: 160, E_pot: -474.68616
[MindSPONGE] Step: 170, E_pot: -495.46088
[MindSPONGE] Step: 180, E_pot: -511.44495
[MindSPONGE] Step: 190, E_pot: -524.12225
[MindSPONGE] Step: 200, E_pot: -534.15643
[MindSPONGE] Step: 210, E_pot: -540.4188
[MindSPONGE] Step: 220, E_pot: -544.38464


In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.coordinate.shape, system.atom_mass)

In [10]:
opt = UpdaterMD(system,
                integrator=VelocityVerlet(system),
                thermostat=Langevin(system, temp),
                velocity=velocity,
                time_step=5e-4)

## 外加限制

在MindSPONGE的WithEnergyCell中可以自定义一些外加的势能项，这是一个非常灵活的操作，支持同时定义多个Bias叠加使用，并且在MindSpore框架的支持下都是可微的。比如本案例中施加的Bias是中心球形势SphericalRestrict，用于把原子的运动限制在一个球形范围内。另外MindSponge也支持原位谐振子势OscillatorBias，可以将原子限制在原始位置附近，这个功能在蛋白质预测中能够起到较大的作用。基于MindSPONGE的基础类PotentialCell，用户还可以自定义更多种类的Bias。

In [11]:
sim = WithEnergyCell(system, potential, bias=SphericalRestrict(radius=1.5, center=[0, 0, 0]))
md = Sponge(sim, optimizer=opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b04.h5md', save_freq=10, write_velocity=True, write_force=True)

In [13]:
md.run(3000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-03-25 16:49:06
[MindSPONGE] Step: 0, E_pot: -563.37964, E_kin: 118.28821, E_tot: -445.09143, Temperature: 270.9867
[MindSPONGE] Step: 10, E_pot: -536.4233, E_kin: 90.163925, E_tot: -446.25934, Temperature: 206.55672
[MindSPONGE] Step: 20, E_pot: -528.2834, E_kin: 80.49756, E_tot: -447.78583, Temperature: 184.41202
[MindSPONGE] Step: 30, E_pot: -506.68964, E_kin: 67.523865, E_tot: -439.16577, Temperature: 154.69055
[MindSPONGE] Step: 40, E_pot: -503.9311, E_kin: 67.37415, E_tot: -436.55695, Temperature: 154.34758
[MindSPONGE] Step: 50, E_pot: -492.82327, E_kin: 56.61204, E_tot: -436.21124, Temperature: 129.69264
[MindSPONGE] Step: 60, E_pot: -494.25098, E_kin: 59.436188, E_tot: -434.8148, Temperature: 136.16249
[MindSPONGE] Step: 70, E_pot: -486.6295, E_kin: 57.96115, E_tot: -428.66833, Temperature: 132.78333
[MindSPONGE] Step: 80, E_pot: -493.23245, E_kin: 64.092995, E_tot: -429.13947, Temperature: 146.83078
[MindSPONGE] Step: 90, E_pot: -490.346